# NLP Project, Classification of Amazon Reviews and Key Phrases
#### CSCI 3832 Natural Language Processing
Members: Adam Wuth, Benjamin Kohav, Noah Vilas, Aiden Devine, Evan Zachary

### Requirements

In [1]:
!pip install torch
!pip install nltk
!pip install datasets
import os, random, sys, copy
import torch, torch.nn as nn, numpy as np
from tqdm.notebook import tqdm
from torch.nn.utils.rnn import pack_padded_sequence, pad_packed_sequence
from nltk.tokenize import word_tokenize
from datasets import load_dataset, concatenate_datasets, load_from_disk
from datetime import datetime

### Load in the data set
The dataset is split into categories, but we wanted all categories from 2020 onwards. This code block will take forever to run, only run it the first time to get the dataset. 

In [19]:
#The dataset is split into categories

categories = [
    "All_Beauty",
    "Amazon_Fashion",
    "Appliances",
    "Arts_Crafts_and_Sewing",
    "Automotive",
    "Baby_Products",
    "Beauty_and_Personal_Care",
    "Books",
    "CDs_and_Vinyl",
    "Cell_Phones_and_Accessories",
    "Clothing_Shoes_and_Jewelry",
    "Digital_Music",
    "Electronics",
    "Gift_Cards",
    "Grocery_and_Gourmet_Food",
    "Handmade_Products",
    "Health_and_Household",
    "Health_and_Personal_Care",
    "Home_and_Kitchen",
    "Industrial_and_Scientific",
    "Kindle_Store",
    "Magazine_Subscriptions",
    "Movies_and_TV",
    "Musical_Instruments",
    "Office_Products",
    "Patio_Lawn_and_Garden",
    "Pet_Supplies",
    "Software",
    "Sports_and_Outdoors",
    "Subscription_Boxes",
    "Tools_and_Home_Improvement",
    "Toys_and_Games",
    "Video_Games",
    "Unknown"
]

#to get reviews from 2023 onwards 2020 onwards was millions of reviews and was taking
#over an hour just to load the data
start_timestamp = int(datetime(2023, 1, 1).timestamp() * 1000)

#to store all datasets
allcats = []

for cat in categories:
    print(f"Loading category: {cat}")
    dataset = load_dataset("McAuley-Lab/Amazon-Reviews-2023", f"raw_review_{cat}", split="full[:150000]",  trust_remote_code=True)
    #dataset = load_dataset("McAuley-Lab/Amazon-Reviews-2023", f"raw_review_{cat}", split="full[:1%]",  trust_remote_code=True)
   #dataset = load_dataset("McAuley-Lab/Amazon-Reviews-2023", cat, split="full",  trust_remote_code=True) formatting issues
    #get the 2023 onwards and add to data
    filtered_dataset = dataset.filter(lambda x: x['timestamp'] >= start_timestamp)
    #allcats.append(dataset)
    allcats.append(filtered_dataset)
#make one final dataset    
reviews = concatenate_datasets(allcats)

print(f"Total reviews loaded: {len(reviews)}")

reviews.save_to_disk("filetred_amazon_reviews")


Loading category: All_Beauty


Filter:   0%|          | 0/150000 [00:00<?, ? examples/s]

Loading category: Amazon_Fashion


Filter:   0%|          | 0/150000 [00:00<?, ? examples/s]

Loading category: Appliances


...

Filter:   0%|          | 0/150000 [00:00<?, ? examples/s]

Loading category: Unknown


Filter:   0%|          | 0/150000 [00:00<?, ? examples/s]

Total reviews loaded: 120088


Saving the dataset (0/1 shards):   0%|          | 0/120088 [00:00<?, ? examples/s]

If you have run that already, reviews was saved(should be in the working directory)so you can just do the next code block instead

In [2]:
reviews = load_from_disk("filetred_amazon_reviews")
print(len(reviews))
print(reviews[0])
print(reviews[1])
print(reviews.column_names)

120088
{'rating': 1.0, 'title': 'halo hair extensions', 'text': "This halo hair extension is simply put, garbage.  Now, you get what you pay for.  And this is a very cheap version.  The faux hair is very shiny and looks literally like bad barbie hair.  It looks WAY better in the photos than in real life.  The color is horrific, in my opinion of course.  The streaks are like paint strips.  And all of that would be one thing - but the worst is that the hair completely fell out!  I had hand fulls of hair strands just trying to put the halo on!  And you might think - well, maybe a little loss is to be expected?  Except this was handfuls and handfuls.  I literally dropped the whole thing right into the trash.  I would say this one is a pass for hair loss alone.  Having said all of this, I never hesitate to update my reviews should new info seem useful. All of my reviews reflect my honest, personal experience with the reviewed item - your experience may be different. I am not influenced by a

### Load in the Glove Embeddings

In [3]:
glove_file = '../glove.6B.50d.txt' # modify to appropriate path for your file system

embeddings_dict = {}

with open(glove_file, 'r', encoding='utf8') as f:
    for i, line in enumerate(f):
        line = line.strip().split(' ')
        word = line[0]
        embed = np.asarray(line[1:], "float")

        embeddings_dict[word] = embed


print('Loaded {} words from glove'.format(len(embeddings_dict)))

low = -1.0 / 3
high = 1.0 / 3
embedding_matrix = np.random.uniform(low=low, high=high, size=(len(embeddings_dict)+1, 50))

word2id = {}
for i, word in enumerate(embeddings_dict.keys(), 1):

    word2id[word] = i                                
    embedding_matrix[i] = embeddings_dict[word]      

word2id['<pad>'] = 0

Loaded 400000 words from glove


### Set up train and validation datasets

In [11]:
#modified from the HW_3 
class RNNMovieReviewDataset(torch.utils.data.Dataset):
    def __init__(self, hf_dataset=None, word2id=None, finalized_data=None, data_limit=250, max_length=128):
        """
        :param hf_dataset: A Hugging Face Dataset object (preloaded and filtered)
        :param word2id: The GloVe word2id dictionary
        :param finalized_data: Used to create validation set
        :param data_limit: Max number of examples to use
        :param max_length: Max sequence length
        """
        self.data_limit = data_limit
        self.max_length = max_length
        self.word2id = word2id

        if finalized_data:
            self.data = finalized_data
        else:
            examples = []
            labels = []

            for i, example in enumerate(hf_dataset):
                if i >= self.data_limit:
                    break
                examples.append(example["text"])
                labels.append(int(example["rating"]) - 1)  # 1–5 stars → 0–4

            tokenized = self.tokenize(examples)
            self.data = [(ids, length, label) for (ids, length), label in zip(tokenized, labels)]
            random.seed(42)
            random.shuffle(self.data)

    def tokenize(self, examples):
        example_ids = []
        misses = 0
        total = 0
        for example in tqdm(examples):
            tokens = word_tokenize(example)
            ids = []
            for tok in tokens:
                if tok in self.word2id:
                    ids.append(self.word2id[tok])
                else:
                    misses += 1
                    ids.append(self.word2id.get('unk', 0))
                total += 1
            
            if len(ids) == 0:
                continue
            
            if len(ids) >= self.max_length:
                ids = ids[:self.max_length]
                length = self.max_length
            else:
                length = len(ids)
                ids += [self.word2id['<pad>']] * (self.max_length - len(ids))

            example_ids.append((torch.tensor(ids), length))

        print(f'Missed {misses} out of {total} words -- {misses/total:.2%}')
        return example_ids

    def generate_validation_split(self, ratio=0.8):
        split_idx = int(ratio * len(self.data))
        val_split = self.data[split_idx:]
        self.data = self.data[:split_idx]
        return val_split

    def __getitem__(self, index):
        return self.data[index]  # returns (input_ids, length, label)

    def __len__(self):
        return len(self.data)


In [12]:
#also modified from hw3
train_dataset = RNNMovieReviewDataset(hf_dataset=reviews, word2id=word2id, data_limit=100000)
validation_examples = train_dataset.generate_validation_split()
print('Loaded {} train examples'.format(len(train_dataset)))

valid_dataset = RNNMovieReviewDataset(finalized_data=validation_examples, word2id=word2id)
print('Loaded {} validation examples'.format(len(valid_dataset)))

print(valid_dataset[0])  # (input_ids, length, label)

  0%|          | 0/100000 [00:00<?, ?it/s]

Missed 656201 out of 5940281 words -- 11.05%
Loaded 79924 train examples
Loaded 19982 validation examples
(tensor([201535,   2556,      2,   2041,   3273,  35268,      2,    841,   5006,
             3, 201535,   4239,  10016,     11,   1282,    416, 201535,  19796,
         30411,    275,  12258, 201535,      4,      8,    144,   5702,   7936,
          1931,      2,     64,     16,  73976,   2756,      3, 201535,    863,
          2401,      2,     21,     15,      8,   7936,   2366,     35,     21,
         66020,   3072,   2160,   9787,     18,      1,    884,     23,     65,
             3,      0,      0,      0,      0,      0,      0,      0,      0,
             0,      0,      0,      0,      0,      0,      0,      0,      0,
             0,      0,      0,      0,      0,      0,      0,      0,      0,
             0,      0,      0,      0,      0,      0,      0,      0,      0,
             0,      0,      0,      0,      0,      0,      0,      0,      0,
             

In [13]:
input_ids, length, label = valid_dataset[1]
print("True length:", length)
print("Non-padded input:", input_ids[:length])
print("Label:", label)


True length: 34
Non-padded input: tensor([201535,     15,      1,   4054,     80,     42,     34,   3169,      1,
          8378,  22645,   3263,      3, 201535,   3388,      8,    531,      6,
           159,   3263,    276,      5,    579,    193,  18133,   2432,      3,
        201535,     87,  31287,    102,   1690,    457,      3])
Label: 4


In [14]:
class LSTMModel(nn.Module):

    def __init__(self, embedding_matrix, lstm_hidden_size=50, num_lstm_layers=1, bidirectional=True, stats=False):

        super().__init__()
        self.embedding = nn.Embedding.from_pretrained(torch.FloatTensor(embedding_matrix))
        self.lstm = nn.LSTM(input_size = embedding_matrix.shape[1],
                            hidden_size = lstm_hidden_size,
                            num_layers = num_lstm_layers,
                            bidirectional = bidirectional,
                            batch_first = True)
        
        self.hidden_1 = nn.Linear(lstm_hidden_size * 2, lstm_hidden_size)
        self.hidden_2 = nn.Linear(lstm_hidden_size, 5)
        self.num_directions = 2 if bidirectional else 1
        self.relu = nn.ReLU()
        self.stats = stats

    def forward(self, input_batch, input_lengths):
        
        if self.stats: print("input_batch: ", input_batch.shape)
        
        embedded_input = self.embedding(input_batch)
        if self.stats: print("embedded_input:", embedded_input.shape)
        
        packed_input = pack_padded_sequence(embedded_input, input_lengths, batch_first=True, enforce_sorted=False)

        packed_output, (hn, cn) = self.lstm(packed_input) # See docs linked below for description of hn.shape
        if self.stats: print("hn:", hn.shape)
        
        hn_view = hn.view(self.lstm.num_layers, self.num_directions, input_batch.shape[0], self.lstm.hidden_size)               # Reshape hn for clarity -- first dimension now represents each layer (total set by num_lstm_layers)
        if self.stats: print("hn_view:", hn_view.shape)
        
        hn_view_last_layer = hn_view[-1]                                                                                        # Taking the last layer for our final LSTM output
        if self.stats: print("hn_view_last_layer:", hn_view_last_layer.shape)
        
        hn_cat = torch.cat([hn_view_last_layer[-2, :, :], hn_view_last_layer[-1, :, :]], dim=1)                                 # Each layer has two directions. We want to use both of these vectors, so concatenate them
        if self.stats: print("hn_cat:", hn_cat.shape)
        
        hid = self.relu(self.hidden_1(hn_cat))
        if self.stats: print("hid:", hid.shape)
        
        output = self.hidden_2(hid)
        if self.stats: print("output:", output.shape)
        
        # raise KeyError
        
        return output

removed sigmoid because 5 classes

In [15]:
def predict(model, valid_dataloader):
    model.eval()
    total_correct = 0
    total_examples = len(valid_dataloader.dataset)

    with torch.no_grad():
        for (x, x_lengths), y in valid_dataloader:
            logits = model(x, x_lengths)               # Shape: [batch_size, 5]
            preds = torch.argmax(logits, dim=1)        # Pick highest logit class
            total_correct += (preds == y).sum().item()

    accuracy = total_correct / total_examples
    print(f'Accuracy: {accuracy:.4f}')
    return accuracy

Changed BCEWithLogitsLoss() to CrossEntropyLoss()

In [16]:
def train_lstm_classification(model, train_dataset, valid_dataset, epochs=10, batch_size=32, learning_rate=.001, print_frequency=25):

    criteria = nn.CrossEntropyLoss()
    optimizer = torch.optim.AdamW(model.parameters(), lr=learning_rate)

    #We'll create an instance of a torch dataloader to collate our data. This class handles batching and shuffling (should be done each epoch)
    train_dataloader = torch.utils.data.DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
    valid_dataloader = torch.utils.data.DataLoader(valid_dataset, batch_size=128, shuffle=True)

    print('Total train batches: {}'.format(train_dataset.__len__() / batch_size))

    best_accuracy = 0.0
    best_model_sd = None

    for i in range(epochs):
        print('### Epoch: ' + str(i+1) + ' ###')
    
        model.train()

        avg_loss = 0

        for step, data in enumerate(tqdm(train_dataloader, desc=f"Training Epoch {i+1}")):

            x, x_lengths, y = data	# Our dataset is returning the input example x and also the lengths of the examples, so we'll unpack that here

            optimizer.zero_grad()

            model_output = model(x, x_lengths)

            loss = criteria(model_output, y)

            loss.backward()
            optimizer.step()

            avg_loss += loss.item()

            if step % print_frequency == (print_frequency - 1):
                print('epoch: {} batch: {} loss: {}'.format(
                    i,
                    step,
                    avg_loss / print_frequency
                ))
                avg_loss = 0

        print('Evaluating...')
        model.eval()
        with torch.no_grad():
            acc = predict(model, valid_dataloader)
            if acc > best_accuracy:
                best_model_sd = copy.deepcopy(model.state_dict())
                best_accuracy = acc

    return model.state_dict(), best_model_sd

In [ ]:
model = LSTMModel(embedding_matrix, lstm_hidden_size=50, num_lstm_layers=2, bidirectional=True)

final_model_state, best_model_state = train_lstm_classification(
    model,
    train_dataset,
    valid_dataset,
    epochs=5,             
    batch_size=128,
    learning_rate=1e-3,
    print_frequency=100
)

Total train batches: 624.40625
### Epoch: 1 ###


Training Epoch 1:   0%|          | 0/625 [00:00<?, ?it/s]